# GitHubAgent runtime test

This notebook performs a runtime test of `doppiozero.agents.gh_deep_search.GitHubAgent`.
By default the notebook will use your real LLM and content layer when available. To force deterministic offline behavior set `USE_MOCKS = True` in the mocking cell.

Run the cells in order. The notebook saves a single `agent_test_output.json` artifact when complete.

In [1]:
# Ensure the repository root is on sys.path so `doppiozero` imports resolve
import sys, os
from pathlib import Path
PROJECT_ROOT = Path(os.getcwd()).resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
print('Inserted PROJECT_ROOT to sys.path:', PROJECT_ROOT)

Inserted PROJECT_ROOT to sys.path: /Users/romanofoti/romanofoti/doppiozero


In [2]:
import types, json
from pprint import pprint
import doppiozero.contents as contents_mod

from doppiozero.agents.gh_deep_search import GitHubAgent

In [ ]:
# Load environment variables and reinitialize the package-level llm_client
# so the agent will use the real LLM (credentials should be in .env).
from dotenv import load_dotenv
load_dotenv()
import importlib
import doppiozero.clients.llm as llm_mod
# Recreate the module-level client using environment credentials
llm_mod.llm_client = llm_mod.LLMClient(verbose=True)
print('Reinitialized doppiozero.clients.llm.llm_client with real LLM (verbose=True)')

In [ ]:
# Mocks removed: this notebook will use the real content layer and LLM
print('Mocks removed; using real content layer and LLM (if configured)')

## Monkeypatch local content layer (mocked integration)

This cell replaces `content_manager` and `content_fetcher` helpers with deterministic functions so the agent can run end-to-end without external services.

In [ ]:
# Real content mode: no mocks or monkeypatching are applied.
# Ensure the real content modules are available to the agent.
cm = getattr(contents_mod, 'content_manager', None)
cf = getattr(contents_mod, 'content_fetcher', None)
print('content_manager present:', cm is not None)
print('content_fetcher present:', cf is not None)
# If you need to initialize real content managers, do so in your environment
# or modify this cell to instantiate them before running the agent.

Patched content_manager and content_fetcher with deterministic mocks


In [4]:
# Interactive clarifying answers (edit this cell before running the agent)
from pathlib import Path

clarifying_answers = '''Q: What is the main goal?
A: Find recent authentication failure discussions across repos.

Q: Are there specific repos to focus on?
A: doppiozero and financial-planning.
'''

clarifying_path = Path('clarifying_answers.txt').resolve()
clarifying_path.write_text(clarifying_answers, encoding='utf-8')
print('Wrote clarifying answers to', clarifying_path)
# The agent will read this file if options['clarifying_qa'] points to it.
# If you prefer to answer interactively, edit the clarifying_answers string above and re-run this cell.

Wrote clarifying answers to /Users/romanofoti/romanofoti/doppiozero/clarifying_answers.txt


## Run the agent (mocked)

Instantiate `GitHubAgent` with lightweight options and run it. The mocked content layer will return deterministic data so the run is fast and offline.

Interactive notes: Edit the clarifying-answers cell (above) if you want to provide clarifying answers manually. If `clarifying_answers.txt` exists when you run the agent cell, it will be used; otherwise the agent will run with no clarifications (or will open an editor if it is configured to do so).

In [5]:
# Run the agent with mocked content layer

from pathlib import Path
# If the clarifying answers file was written above, prefer it; otherwise leave as None
clarifying_file = Path('clarifying_answers.txt').resolve()
clarifying_qa = str(clarifying_file) if clarifying_file.exists() else None

options = {
    'collection': None,
    'limit': 3,
    'max_depth': 1,
    'editor_file': None,
    # Use clarifying_qa if the file exists; set to None to allow interactive editing
    'clarifying_qa': clarifying_qa,
    'search_modes': ['semantic'],
    'cache_path': None,
    'models': {'fast': 'default', 'reasoning': 'default', 'embed': 'default'},
    'parallel': False,
    'verbose': True,
}
agent = GitHubAgent('What are the recent discussions about authentication failures?', options)
result = agent.run()
print('=== AGENT RUN RESULT ===')
pprint(result)
print('=== SHARED STATE ===')
pprint(agent.shared)

2025-09-04 15:49:07,069 - doppiozero.agents.gh_deep_search - gh_deep_search.py  - run          - INFO     - === GITHUB CONVERSATIONS RESEARCH AGENT ===
2025-09-04 15:49:07,071 - doppiozero.agents.gh_deep_search - gh_deep_search.py  - run          - INFO     - Request: What are the recent discussions about authentication failures?
2025-09-04 15:49:07,072 - doppiozero.agents.gh_deep_search - gh_deep_search.py  - run          - INFO     - Collection: None
2025-09-04 15:49:07,072 - doppiozero.agents.gh_deep_search - gh_deep_search.py  - run          - INFO     - Max results per search: 3
2025-09-04 15:49:07,072 - doppiozero.agents.gh_deep_search - gh_deep_search.py  - run          - INFO     - Max deep research iterations: 1
2025-09-04 15:49:07,073 - doppiozero.agents.gh_deep_search - gh_deep_search.py  - run          - INFO     - Fast model: default
2025-09-04 15:49:07,073 - doppiozero.agents.gh_deep_search - gh_deep_search.py  - run          - INFO     - Reasoning model: default
2025-09-

=== AGENT RUN RESULT ===
{'claims_verified': 0,
 'draft': '{}',
 'num_conversations': 13,
 'unsupported_claims': ['Claim 1', 'Claim 2']}
=== SHARED STATE ===
{'cache_path': None,
 'claim_verification': {'details': [{'claim': 'Claim 1',
                                     'evidence': [{'score': 1.0,
                                                   'snippet': 'Summary of '
                                                              'Claim 1 #0',
                                                   'source': 'https://example.com/convo/0'},
                                                  {'score': 0.9,
                                                   'snippet': 'Summary of '
                                                              'Claim 1 #1',
                                                   'source': 'https://example.com/convo/1'},
                                                  {'score': 0.8,
                                                   'snippet': 'Summary of '
   

2025-09-04 15:46:20,238 - doppiozero.agents.gh_deep_search - gh_deep_search.py  - run          - INFO     - Collection: None


2025-09-04 15:46:20,238 - doppiozero.agents.gh_deep_search - gh_deep_search.py  - run          - INFO     - Max results per search: 3


2025-09-04 15:46:20,239 - doppiozero.agents.gh_deep_search - gh_deep_search.py  - run          - INFO     - Max deep research iterations: 1


...

2025-09-04 15:46:20,250 - doppiozero.nodes.reporter - reporter.py        - post         - INFO     - 1. Claim 1


2025-09-04 15:46:20,250 - doppiozero.nodes.reporter - reporter.py        - post         - INFO     - 2. Claim 2


2025-09-04 15:46:20,251 - doppiozero.nodes.reporter - reporter.py        - post         - INFO     - 

✓ Research complete! Total conversations analyzed: 13, 2 claims verified (0 supported, 2 unsupported)


2025-09-04 15:46:20,251 - doppiozero.nodes.end - end.py             - exec         - INFO     - End node: terminating workflow and returning results.


=== AGENT RUN RESULT ===
{'claims_verified': 0,
 'draft': '{}',
 'num_conversations': 13,
 'unsupported_claims': ['Claim 1', 'Claim 2']}
=== SHARED STATE ===
{'cache_path': None,
 'claim_verification': {'details': [{'claim': 'Claim 1',
                                     'evidence': [{'score': 1.0,
                                                   'snippet': 'Summary of '
                                                              'Claim 1 #0',
                                                   'source': 'https://example.com/convo/0'},
                                                  {'score': 0.9,
                                                   'snippet': 'Summary of '
                                                              'Claim 1 #1',
                                                   'source': 'https://example.com/convo/1'},
                                                  {'score': 0.8,
                                                   'snippet': 'Summary of '
   

In [6]:
# Simple runtime checks (no external test files created)
assert isinstance(result, dict), 'Agent result should be a dict'
# The agent returns a structured final report placed in shared['final_report']
assert 'num_conversations' in result, 'Result missing num_conversations key'
print('Basic assertions passed: result appears to be a final report dict')

Basic assertions passed: result appears to be a final report dict


In [7]:
# Save the result artifact for inspection
import json
out_path = Path('agent_test_output.json').resolve()
with open(out_path, 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=2)
print('Saved agent output to', out_path)

Saved agent output to /Users/romanofoti/romanofoti/doppiozero/agent_test_output.json


## Next steps / integration notes

- To run the agent against real GitHub data, remove or replace the monkeypatch cell and set real `collection`, `cache_path` and credentials via environment variables (do not commit tokens).
- If you want me to add optional assertions on `agent.shared` (for example `claim_verification`), tell me which keys you expect and I'll add them.
- If anything in this offline run fails, paste the notebook output and I will debug the failing cell.